# Condor Submit Explanation

First step is to run the script `condor_submit_nano_connect_ntuples.py`, give this script a `.list` file as one of the inputs, for example:

`[dgrove@login SUSYCascades]$ python3 scripts/condor_submit_nano_connect_ntuples.py -list samples/NANO/Lists/SUSYCascade_Sample.list -split 20 --verbose --dry-run`

# IF RUNNING OVER SIGNAL:

`[dgrove@login SUSYCascades]$ python3 scripts/condor_submit_nano_connect_ntuples.py -list samples/NANO/Lists/SUSYCascade_Sample.list -split 20 --verbose --dry-run --sms`


**Note:** This is run from the SUSYCascades directory

output the `.list` file to see what it looks like:

```
[dgrove@login SUSYCascades]$ cat samples/NANO/Lists/Summer23_130X.list
samples/NANO/Summer23_130X/TTto2L2Nu_TuneCP5_13p6TeV_powheg-pythia8.txt
```

Lets go into the `.txt` now and see whats in there:


`[dgrove@login SUSYCascades]$ cat samples/NANO/Summer23_130X/TTto2L2Nu_TuneCP5_13p6TeV_powheg-pythia8.txt`
```
root://cmsxrootd.fnal.gov//store/mc/Run3Summer23NanoAODv12/TTto2L2Nu_TuneCP5_13p6TeV_powheg-pythia8/NANOAODSIM/130X_mcRun3_2023_realistic_v14-v2/70000/428c6754-4ad2-4c2f-8ec6-4f2b5dc60758.root
root://cmsxrootd.fnal.gov//store/mc/Run3Summer23NanoAODv12/TTto2L2Nu_TuneCP5_13p6TeV_powheg-pythia8/NANOAODSIM/130X_mcRun3_2023_realistic_v14-v2/70000/d48cf12d-e6f4-4c59-ba57-126ee52c18a0.root
...
```

... implies many many more redirectors to different root files. These are all the root files in a given MC data sample, seen [here](https://cmsweb.cern.ch/das/request?instance=prod/global&input=file+dataset%3D%2FTTto2L2Nu_TuneCP5_13p6TeV_powheg-pythia8%2FRun3Summer23NanoAODv12-130X_mcRun3_2023_realistic_v14-v2%2FNANOAODSIM)

So, lets recap, we need a `.txt` file which is full of all the redirector links to the root files we want to run over. Next, we need a `.list` file that simply paths to the location of that `.txt` file (a `.list` file can have multiple paths on separate lines so we can have many `.txt` file inputs (which would correspond to multiple MC data samples for background or signal or whatever) into a single condor submit script)

Now then, the output of the `condor_submit_nano_connect_ntuples.py` script is a directory that contains the relevant files and logs to submit to condor. To submit to condor we run:

`condor_submit` and as an input we give it the location of the `.submit` file generated from the python script, full command example would be this:

`condor_submit Summer23_130X/src/TTto2L2Nu_TuneCP5_13p6TeV_powheg-pythia8_Summer23_130X_single.submit`

here there would be two `.submit` files at that location, 

`TTto2L2Nu_TuneCP5_13p6TeV_powheg-pythia8_Summer23_130X_single.submit`

and

`TTto2L2Nu_TuneCP5_13p6TeV_powheg-pythia8_Summer23_130X.submit`

the difference being, `~_single.submit` is meant to submitted first as a test to see if what comes back from condor looks correct. If that comes back ok then it should be fine to submit the full job, `TTto2L2Nu_TuneCP5_13p6TeV_powheg-pythia8_Summer23_130X.submit`

Once the job is submitted to condor we can keep tabs with 

`watch condor_q $USER -batch`

The output files should be in a subdirectory in the `/ospool/cms-user/dgrove/NTUPLES/` area

I've been working in this directory:
`/eos/home-d/dgrove/datasets` on lxplus



## Nuance (understand this before trying to make `.submit` files)

the condor submit python file has a list named `knowntags` that it needs to have an entry for the `.list` file in order to find said list file, I think its for the list file at least. 


Caleb in Slack: "A common problem is getting held jobs for a number of CPU issue… I have examples in my text files, but I will include this if I make instructions…"

`condor_q -held`

```
-- Schedd: login-el7.uscms.org : <192.170.231.212:9618?... @ 04/18/24 10:45:37
 ID          OWNER          HELD_SINCE  HOLD_REASON
17067736.446 caleb           4/18 10:43 The number of CPU load exceeds the number of requested CPUs. Please, check your workflow c
pu/core requirements.        
17067736.455 caleb           4/18 10:44 The number of CPU load exceeds the number of requested CPUs. Please, check your workflow cpu/core requirements.        
17067736.456 caleb           4/18 10:43 The number of CPU load exceeds the number of requested CPUs. Please, check your workflow cpu/core requirements.
```

For this problem, we do these two commands, which Zach provided:

`condor_qedit $USER RequestCpus=4`
```
Set attribute "RequestCpus" for 154 matching jobs.
````

`condor_release -all`
```
All jobs have been released
```

## Generating text files conveniently:

From Caleb, do these commands to generate the text files that we need for our condor job (.list file is still manually created however its just a directory pointer to this text file). You will need grid certificates and proxy enabled for this to work:

`dasgoclient -query="dataset=/TTJets_DiLept_TuneCP5_13TeV-madgraphMLM-pythia8/RunIISummer20UL17NanoAODv9-106X_mc2017_realistic_v9-v1/NANOAODSIM"`

That queries the server for the data set?

`dasgoclient -query="file dataset=/TTJets_DiLept_TuneCP5_13TeV-madgraphMLM-pythia8/RunIISummer20UL17NanoAODv9-106X_mc2017_realistic_v9-v1/NANOAODSIM"`

The above does almost the same but it prints out all the files for that data set. Can you guess the next step? Pipe it into a .txt file:

`dasgoclient -query="file dataset=/TTJets_DiLept_TuneCP5_13TeV-madgraphMLM-pythia8/RunIISummer20UL17NanoAODv9-106X_mc2017_realistic_v9-v1/NANOAODSIM" > TTJets_DiLept_TuneCP5_13TeV-madgraphMLM-pythia8_UL2017_NanoAODv9.txt`

We are missing one final step, we need to append to the beginning of each line the text for our redirector, so run this:

`sed -i 's/^/root:\/\/cms-xrd-global.cern.ch\//' TTJets_DiLept_TuneCP5_13TeV-madgraphMLM-pythia8_UL2017_NanoAODv9.txt`

Kepe in mind you need to change the text file name for something more appropriate relative to the dataset you're printing, then you also need to use that same text file name in the last command.

For checking *used* storage space on cms connect:

```
alias check_home='du -sh /home/$USER'

alias check_ospool='du -sh /ospool/cms-user/$USER'

alias check_scratch='du -sh /local-scratch/$USER'
```

Useful commands for editing condor jobs after: 

`condor_qedit`

`condor_qedit $USER RequestMemory=1000`